In [7]:
# install datasets library
!uv pip install datasets
!uv pip install python-dotenv

Using Python 3.12.3 environment at: /home/jake/git/vllm-chatbot/.venv
Audited 1 package in 3ms
Using Python 3.12.3 environment at: /home/jake/git/vllm-chatbot/.venv
Audited 1 package in 1ms


In [8]:
import os
from dotenv import load_dotenv

load_dotenv(".env")

True

# Load a csv file to generate dataset

In [9]:
import pandas as pd

df = pd.read_csv('movensys_web_info_2508.csv', encoding='latin1')
df.head()

,Question,Answer
0,What kind of company is Movensys and what is i...,Movensys is an IT company that provides total ...
1,What are the main product categories offered b...,Movensys offers three main product categories:...
2,What is the core philosophy or mission behind ...,Movensys' core philosophy is centered on provi...
3,What is Movensys' physical address and primary...,"Movensys is located at 8F Joy City Tower, 25 H..."
4,What is Movensys' primary focus and core busin...,Movensys provides Software Motion Control solu...


In [10]:
# loaded lines
len(df)

23

# Generate QA Pair

In [11]:
qa_pair = []

for _, row in df.iterrows():
    qa_pair.append({'question': row['Question'], 'answer': row['Answer']})

In [12]:
qa_pair

[{'question': 'What kind of company is Movensys and what is its primary focus?',
  'answer': 'Movensys is an IT company that provides total motion control solutions for factory automation. It is also described as a software-based controller platform provider.'},
 {'question': 'What are the main product categories offered by Movensys?',
  'answer': 'Movensys offers three main product categories: Software Motion Control, which specifically includes WMX and WMX Extension. This is described as a software-based motion control platform. Hardware Solution, which provides a variety of solutions such as IPC, IO, Servo, Step, and Module. Navigation Software for AMR (Autonomous Mobile Robot), referred to as AMR Solution. This is a total AMR control platform.'},
 {'question': "What is the core philosophy or mission behind Movensys' technology?",
  'answer': "Movensys' core philosophy is centered on providing more innovative value beyond motion control through its open architecture-based technology

In [13]:
import json

with open("qa_pair.jsonl", "w", encoding="utf-8") as f:
   # Write each QA pair as a JSON object on a new line
    for pair in qa_pair:
        f.write(json.dumps(pair, ensure_ascii=False) + "\n")

# Load the json file as a Hugging Face Datasets format

In [14]:
from datasets import load_dataset

# load the jsonl file
dataset = load_dataset("json", data_files="qa_pair.jsonl")

/home/jake/git/vllm-chatbot/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 23 examples [00:00, 7656.88 examples/s]


In [15]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 23
    })
})

In [16]:
from huggingface_hub import HfApi

# Create a HfApi instance
api = HfApi()

# repo to updload this datset
repo_name = "Jake5/movensys-info"
token = os.getenv("HF_TOKEN")

# Push the dataset to the Hugging Face Hub
dataset.push_to_hub(repo_name, token=token)

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.37s/ shards]


CommitInfo(commit_url='https://huggingface.co/datasets/Jake5/movensys-info/commit/f80080c500a27c63f5344e183c19f50bfe32622c', commit_message='Upload dataset', commit_description='', oid='f80080c500a27c63f5344e183c19f50bfe32622c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Jake5/movensys-info', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Jake5/movensys-info'), pr_revision=None, pr_num=None)